# <font color=green>produce visuals, put into functions, module, dash app</font>

## <font color=red> check closed on time, and ensure future runs don't change historical indicator values by not only filtering for workorders with notnull on date completed but also filtering for not null and a date completed date that is in the fiscal year in which it was requested</font>

In [137]:
import datadotworld as dw
import numpy as np
import pandas as pd

import plotly as py
import plotly.graph_objs as go
import plotly.offline as pyo

In [138]:
pyo.init_notebook_mode(connected=True)

## fetcher module functions

In [2]:
def fiscal_year(df):
    """
    Function takes a dataframe with a DateTimeIndex and
    produces list with the corresponding fiscal year as a
    four digit year for each date on the index of the dataframe.

    The function is based on the Maryland Govt fiscal year which
    runs from July 1st to June 30th.  It returns a list that is the
    same size as the original dataframe and allows the function call
    to be passed as a new column for fiscal year.
    """
    fiscal_year = np.where(df.index.month >= 7,df.index.year+1, df.index.year)
    return fiscal_year

In [454]:
# load dataset from data.world service
dataset = dw.load_dataset('dgs-kpis/fmd-maintenance')
df = dataset.dataframes['archibus_maintenance_data']

/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/datadotworld/models/dataset.py:194: UserWarning:

Unable to set data frame dtypes automatically using archibus_maintenance_data schema. Data types may need to be adjusted manually. Error: Integer column has NA values in column 50

/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/datadotworld/util.py:118: DtypeWarning:

Columns (6,8,30,32,34,38,49,53,56,60,68,74,83,85,86,88,90,91) have mixed types. Specify dtype option on import or set low_memory=False.



In [455]:
type(dataset)

datadotworld.models.dataset.LocalDataset

In [461]:
target_columns = ['wo_id','date_completed','prob_type',
                   'bl_id','completed_by','time_completed','time_start','time_end']




,wo_id,date_completed,prob_type,bl_id,completed_by,time_completed,time_start,time_end,duration,fiscal_year,fiscal_year_completed
date_requested,,,,,,,,,,,
2018-11-16,111962.0,2018-11-19,OTHER,B00005,ANTHONY.PATTERSON,06:20:28,NaN,NaN,3 days,2019,2019.0
2018-11-16,111961.0,2018-11-19,OTHER,B00005,ANTHONY.PATTERSON,06:21:41,NaN,NaN,3 days,2019,2019.0
2018-11-16,111961.0,2018-11-19,OTHER,B00005,ANTHONY.PATTERSON,06:21:41,NaN,NaN,3 days,2019,2019.0
2018-11-16,111961.0,2018-11-19,OTHER,B00005,ANTHONY.PATTERSON,06:21:41,NaN,NaN,3 days,2019,2019.0
2018-11-16,111960.0,2018-11-19,OTHER,B00005,ANTHONY.PATTERSON,06:24:39,NaN,NaN,3 days,2019,2019.0
2018-11-16,111960.0,2018-11-19,OTHER,B00005,ANTHONY.PATTERSON,06:24:39,NaN,NaN,3 days,2019,2019.0
2018-11-16,111960.0,2018-11-19,OTHER,B00005,ANTHONY.PATTERSON,06:24:39,NaN,NaN,3 days,2019,2019.0
2018-11-16,111963.0,2018-11-20,CARPENTRY,B09864,ANTHONY.PATTERSON,06:09:30,NaN,NaN,4 days,2019,2019.0
2018-11-16,111963.0,2018-11-20,CARPENTRY,B09864,ANTHONY.PATTERSON,06:09:30,NaN,NaN,4 days,2019,2019.0


In [462]:
def get_dataframe(data_source):
    '''
    
    '''
    target_columns = (['wo_id','date_completed','prob_type','bl_id','completed_by',
                        'date_requested','time_completed','time_start','time_end'])
    if isinstance(data_source, str):
        df = pd.read_excel(io=data_source)
        df = df[target_columns][(df['prob_type'] != 'TEST(DO NOT USE)')]
        df['date_requested'] = pd.to_datetime(df['date_requested'])
        df.set_index('date_requested', inplace=True)
        df['duration'] = df['date_completed'] - df.index
        df['fiscal_year_requested'] = fiscal_year(df)  
        df['fiscal_year_completed'] =  [date.year + 1 if date.month >= 7 else date.year for date in df['date_completed']]
        df.sort_index(inplace=True)
        
    elif isinstance(data_source, dw.models.dataset.LocalDataset):
        df = data_source.dataframes['archibus_maintenance_data']
        df = df[target_columns][(df['prob_type'] != 'TEST(DO NOT USE)')]        
        df['date_completed'] = pd.to_datetime(df['date_completed'])
        df['date_requested'] = pd.to_datetime(df['date_requested'])
        df.set_index('date_requested', inplace=True)
        df['duration'] = df['date_completed'] - df.index
        df['fiscal_year_requested'] = np.where(df.index.month >= 7,df.index.year+1, df.index.year)
        df['fiscal_year_completed'] =  [date.year + 1 if date.month >= 7 else date.year for date in df['date_completed']]
        df.sort_index(inplace=True)
        
    else:
        print("""
Function expects type io string or datadotworld.models.dataset.LocalDataset 
but got {}""".format(type(data_source)))
        
    return df
    

,wo_id,date_completed,prob_type,bl_id,completed_by,time_completed,time_start,time_end,duration,fiscal_year
date_requested,,,,,,,,,,
2013-10-16,19.0,2014-03-12,HVAC,B04004,BONNIE.MILLER,11:48:49,NaN,NaN,147 days,2014
2013-10-16,2.0,2013-10-23,FENCE_GATE,B00163,ANN.BRAUN,07:54:25,NaN,NaN,7 days,2014
2013-10-16,16.0,2014-01-08,CEILTILE,B00056,JIMMY.HOLTHAUS,14:08:09,NaN,NaN,84 days,2014
2013-10-16,11.0,2013-10-16,PLUMB/OTHER,B00020,CHARLES.CLARIDY,13:06:44,NaN,NaN,0 days,2014
2013-10-16,20.0,2013-10-23,CEILTILE,B08535,ANTHONY.DINISIO,12:30:12,NaN,NaN,7 days,2014
2013-10-16,6.0,2013-10-16,PLUMB/OTHER,B00021,CHARLES.CLARIDY,13:04:58,NaN,NaN,0 days,2014
2013-10-16,7.0,2013-10-17,PAINT,B06070,ANN.BRAUN,08:11:16,NaN,NaN,1 days,2014
2013-10-16,18.0,2014-03-04,HVAC,B06038,BONNIE.MILLER,14:06:13,NaN,NaN,139 days,2014
2013-10-16,10.0,2013-10-21,PAINT,B06065,ANN.BRAUN,13:34:14,NaN,NaN,5 days,2014


In [463]:
df = get_dataframe(dataset)

In [466]:
fy_list, fy_dfs = [],[]
for year in df['fiscal_year_requested'].unique():
    fy_dfs.append(df.loc[(df['fiscal_year_requested']==year)])
    fy_list.append(year)
fy_dfs = dict(zip(fy_list,fy_dfs))

In [467]:
# get total number of work orders requested in each fiscal year
work_order_volume = []
for key, value in fy_dfs.items():
    work_order_volume.append(value.wo_id.count())
work_order_volume = dict(zip(fy_list,work_order_volume))
work_order_volume

{2014: 12209, 2015: 14522, 2016: 16679, 2017: 17873, 2018: 16593, 2019: 6186}

In [468]:
closed_workorders_dfs = []
for key, value in fy_dfs.items():
    closed_workorders_dfs.append(value[value['date_completed'].notnull()])    
closed_workorders_dfs = dict(zip(fy_dfs.keys(), closed_workorders_dfs))

In [469]:
closed_workorders_dfs[2019].head()

,wo_id,date_completed,prob_type,bl_id,completed_by,time_completed,time_start,time_end,duration,fiscal_year_requested,fiscal_year_completed
date_requested,,,,,,,,,,,
2018-07-01,106281.0,2018-07-01,OTHER,B00061,NaN,13:59:13,NaN,NaN,0 days,2019,2019.0
2018-07-01,106279.0,2018-07-01,OTHER,B00057,NaN,13:45:59,NaN,NaN,0 days,2019,2019.0
2018-07-01,106280.0,2018-07-01,OTHER,B00056,NaN,13:46:50,NaN,NaN,0 days,2019,2019.0
2018-07-01,106276.0,2018-07-11,PREVENTIVE MAINT,B00063,NaN,08:07:34,NaN,NaN,10 days,2019,2019.0
2018-07-01,106284.0,2018-07-05,BATHROOM_FIXT,B00163,NaN,14:03:14,NaN,NaN,4 days,2019,2019.0


In [135]:
# add data for work order volume by problem type per fiscal year
# add average duration for each problem type per fiscal year
# add indicator for whether work order was on time or not

for key, value in closed_workorders_dfs.items():
    value['count'] = value.groupby(['prob_type'])['prob_type'].transform('count')
    value['avg_duration'] = (value.groupby(['prob_type'])['duration'].\
                        transform('sum') / value['count'])
    value['on_time'] = np.where(value.duration <= value.avg_duration,'hit','miss')


/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is 

## pm to cm 

In [755]:
# define filters for calculating PM : CM indicator
corrective_maintenance = ['BOILER','CHILLERS','COOLING TOWERS','HVAC',
                                'HVAC INFRASTRUCTURE','HVAC|REPAIR']
preventative_maintenance = ['PREVENTIVE MAINT','HVAC|PM']
hvac_problem_types = corrective_maintenance + preventative_maintenance

kpi_values_dict = {}
for key, value in fy_dfs.items():
        kpi_values_dict[key] = (
        value[value['prob_type'].isin(preventative_maintenance)]['prob_type'].
        value_counts().sum() / 
    
        value[value['prob_type'].isin(corrective_maintenance)]['prob_type'].
        value_counts().sum() * 100 )

In [756]:
trace1 = go.Bar(
    x = [year for year in kpi_values_dict.keys()],
    y = [value for value in kpi_values_dict.values()],
    marker = dict(color = '#3c5a89'),
    name = 'pm / cm'
                )

trace2 = go.Bar(
    x = [year for year in work_order_volume.keys()],
    y = [value for value in work_order_volume.values()],
    marker = {'color': 'grey',
             'line':{'color': 'black',
                    'width': 1.5}},
    opacity = .2,
    yaxis = 'y2',
    name = 'total work orders'
                )

layout = go.Layout(
        hovermode = 'closest',
        legend = dict(orientation = 'h'),
        title =' HVAC PM:CM KPI<br>FY{} -- FY{}'.format(min([year for year in kpi_values_dict.keys()]),max([year for year in kpi_values_dict.keys()])),
        yaxis=dict(title='pct %',
                   showgrid = False,
                   titlefont = dict(color = '#3c5a89'),
                   tickfont = dict(color = '#3c5a89')),
        yaxis2=dict(title='total work order volume',
                    showgrid = False,
                    titlefont=dict(color='grey'),
            tickfont=dict(
                color='grey'
            ),
            overlaying='y',
            side='right'
        ),
        plot_bgcolor = '#efefea',
        paper_bgcolor = '#efefea'
    )

fig = dict(data=[trace1,trace2], layout=layout)
pyo.iplot(fig)

In [184]:
work_order_volume

{2014: 12209, 2015: 14522, 2016: 16679, 2017: 17873, 2018: 16593, 2019: 6186}

In [207]:
trace1 = go.Scatter(
    x = [year for year in work_order_volume.keys()],
    y = [value for value in work_order_volume.values()],
    mode = 'lines',
    name = 'total work orders'
                    )
layout = go.Layout(
        hovermode = 'closest',
        legend = dict(orientation = 'h'),
        title ='Work Order Volume<br>FY{} -- FY{}'.format(min([year for year in kpi_values_dict.keys()]),max([year for year in kpi_values_dict.keys()])),
        yaxis=dict(title='requests',
                   showgrid = False,
                   titlefont = dict(color = '#3c5a89'),
                   tickfont = dict(color = '#3c5a89')),
    
        plot_bgcolor = '#efefea',
        paper_bgcolor = '#efefea'
    )
fig = dict(data=[trace1], layout=layout)
pyo.iplot(fig)

In [225]:
# plot monthly
trace1 = go.Scatter(
    x = [quarter for quarter in df.resample('M')['wo_id'].count().index],
    y = [value for value in df.resample('M')['wo_id'].count()],
    mode = 'lines',
    name = 'monthly work orders',
    line = {'color': 'blue',
           'width': 3,
           'dash': 'dot'}
                    )
trace2 = go.Scatter(
    x = [year for year in work_order_volume.keys()],
    y = [value for value in work_order_volume.values()],
    mode = 'lines',
    name = 'annual work orders',
    line = {'color': 'red',
           'width': 2},
    yaxis = 'y2')
    
layout = go.Layout(
        hovermode = 'closest',
        legend = dict(orientation = 'h'),
        title ='Work Order Volume<br>FY{} -- FY{}'.format(min(df['fiscal_year']),max(df['fiscal_year'])),
        yaxis = dict(title='requests (monthly)<br>calendar year',
                   showgrid = False,
                   titlefont = dict(color = '#3c5a89'),
                   tickfont = dict(color = '#3c5a89')),
    
        yaxis2=dict(title='requests (annually)<br>fiscal year',
                    showgrid = False,
                    titlefont=dict(color='red'),
                    tickfont=dict(color='red'),
                    overlaying='y',
                    side='right'
                    ),
        plot_bgcolor = '#efefea',
        paper_bgcolor = '#efefea'
    )
fig = dict(data=[trace1, trace2], layout=layout)
pyo.iplot(fig)

In [226]:
# plot quarterly
trace1 = go.Scatter(
    x = [quarter for quarter in df.resample('Q')['wo_id'].count().index],
    y = [value for value in df.resample('Q')['wo_id'].count()],
    mode = 'lines',
    name = 'quarterly work orders',
    line = {'color': 'blue',
           'width': 4,
           'dash': 'dash'}
                    )
trace2 = go.Scatter(
    x = [year for year in work_order_volume.keys()],
    y = [value for value in work_order_volume.values()],
    mode = 'lines',
    name = 'annual work orders',
    line = {'color': 'red',
           'width': 2},
    yaxis = 'y2')
    
layout = go.Layout(
        hovermode = 'closest',
        legend = dict(orientation = 'h'),
        title ='Work Order Volume<br>FY{} -- FY{}'.format(min(df['fiscal_year']),max(df['fiscal_year'])),
        yaxis = dict(title='requests (quarterly)<br>calendar year',
                   showgrid = False,
                   titlefont = dict(color = '#3c5a89'),
                   tickfont = dict(color = '#3c5a89')),
    
        yaxis2=dict(title='requests (annually)<br>fiscal year',
                    showgrid = False,
                    titlefont=dict(color='red'),
                    tickfont=dict(color='red'),
                    overlaying='y',
                    side='right'
                    ),
        plot_bgcolor = '#efefea',
        paper_bgcolor = '#efefea'
    )
fig = dict(data=[trace1, trace2], layout=layout)
pyo.iplot(fig)

In [261]:
# xaxis set month
# plot pct on time, number on time, total volume for fiscal year 
traces = []
red_colors = ['#ff1a1a','#ff4d4d','#ff8080','#cc0000',
           '#ffe6e6','#b30000','#660000']
for year in  closed_workorders_dfs.keys():
    traces.append(
        go.Scatter(
            x = [month.strftime('%b') for month in closed_workorders_dfs[2015].resample('M')['wo_id'].count().index],
            y = [value for value in closed_workorders_dfs[year].resample('M')['wo_id'].count()],
            mode = 'lines',
            name = 'FY{}'.format(year),
            line = {'width': 2},
            yaxis = 'y2'),
                )

layout = go.Layout(
    hovermode = 'closest',
    legend = {'orientation': 'h'},
    title ='Work Orders Closed On Time<br>FY{} -- FY{}'.format(min([year for year in closed_workorders_dfs.keys()]),max([year for year in closed_workorders_dfs.keys()])),
        yaxis = dict(title='work orders closed<br> on time (monthly)',
                   showgrid = False,
                   titlefont = dict(color = '#3c5a89'),
                   tickfont = dict(color = '#3c5a89')),
    
        yaxis2=dict(title='work order volume',
                    showgrid = False,
                    titlefont=dict(color='red'),
                    tickfont=dict(color='red'),
                    overlaying='y',
                    side='right'),
        plot_bgcolor = '#efefea',
        paper_bgcolor = '#efefea'
                    )
fig = {'data':traces,'layout':layout}
pyo.iplot(fig)



In [290]:
closed_workorders_dfs[2018]['on_time'].value_counts()

hit     10898
miss     4389
Name: on_time, dtype: int64

In [294]:
kpi_values_dict = {}
for year in closed_workorders_dfs.keys():
    kpi_values_dict[year] = closed_workorders_dfs[year]['on_time'].\
                            value_counts()[0] / fy_dfs[year]['wo_id'].count() * 100

kpi_values_dict

{2014: 67.17175853878287,
 2015: 78.30877289629528,
 2016: 74.76467414113556,
 2017: 69.79242432719744,
 2018: 65.67829807750256,
 2019: 49.88684125444552}

In [299]:
# plot volume over time y2  and count missed, pct on time, count on time y1

y1_trace = go.Scatter(
    x = [year for year in kpi_values_dict.keys()],
    y = [value for value in kpi_values_dict.values()],
    mode = 'lines',
    name = 'on-time %',
    line = {'color': 'blue'}
)

y2_trace = go.Scatter(
    x = [year for year in work_order_volume.keys()],
    y = [value for value in work_order_volume.values()],
    mode = 'lines',
    name = 'annual work orders',
    line = {'color': 'darkgrey',
           'width': 4},
    yaxis = 'y2')


layout = go.Layout(
    hovermode = 'closest',
    legend = {'orientation': 'h'},
    title = 'On Time Overview',
    yaxis = dict(title = 'work orders closed<br> on time (pct)',
                showgrid = False,
                titlefont = dict(color='blue')),
    
    yaxis2=dict(title='number requests',
                    showgrid = False,
                    titlefont=dict(color='darkgrey'),
                    tickfont=dict(color='darkgrey'),
                    overlaying='y',
                    side='right'),
    plot_bgcolor = '#efefea',
    paper_bgcolor = '#efefea'
                    )
fig ={'data':[y1_trace, y2_trace], 'layout': layout}
pyo.iplot(fig)



# <font color=orange>keeper</font>

In [314]:
# bar charts of pct closed on time vs volume

y1_trace = go.Bar(
    x = [year for year in kpi_values_dict.keys()],
    y = [value for value in kpi_values_dict.values()],
    name = 'on-time %',
    marker = dict(color = '#3c5a89'),
    opacity = .7
                    )

y2_trace = go.Bar(
    x = [year for year in work_order_volume.keys()],
    y = [value for value in work_order_volume.values()],
    name = 'annual work orders',
    marker = {'color': 'grey'},
    opacity = .8,
    yaxis = 'y2')


layout = go.Layout(
    hovermode = 'closest',
    legend = {'orientation': 'h'},
    title = 'On Time Overview',
    xaxis = {'title': 'fiscal year'},
    yaxis = dict(title = 'work orders closed<br> on time (pct)',
                showgrid = False,
                titlefont = dict(color='blue')),
    
    yaxis2=dict(title='number requests',
                    showgrid = False,
                    titlefont=dict(color='darkgrey'),
                    tickfont=dict(color='darkgrey'),
                    overlaying='y',
                    side='right'),
    plot_bgcolor = '#efefea',
    paper_bgcolor = '#efefea'
                    )
fig ={'data':[y1_trace, y2_trace], 'layout': layout}
pyo.iplot(fig)



In [ ]:
# plot number of work orders closed in each month by completed date overtime bar chart

In [351]:
# line graph monthly requests vs closed work orders
trace1 = go.Scatter(
    x = [date for date in df[df['date_completed'].notnull()].resample('M')['date_completed'].count().index],
    y = [value for value in df[df['date_completed'].notnull()].resample('M')['date_completed'].count()],
    name = 'closed',
    mode = 'lines',
    #marker = dict(color = '#3c5a89'),
    opacity = .7
                )
trace2 = go.Scatter(
    x = [date for date in df.resample('M')['wo_id'].count().index],
    y = [value for value in df.resample('M')['wo_id'].count()],
    name = 'requested',
    mode = 'lines',
    #marker = {'color': 'orange'},
    
)

layout = go.Layout(
    hovermode = 'closest',
    title = 'When are Work Orders Closed',
    yaxis = {'title': 'work orders<br>(per month)'},
    xaxis = {'title': 'fiscal year'}
                    )

fig = {'data': [trace1,trace2], 'layout': layout}
pyo.iplot(fig)

## <font color=orange>keeper</font>

In [698]:
# show requested vs closed in month as pct of total requested 
trace1 = go.Bar(
    x = [date for date in df[df['date_completed'].notnull()].resample('M')['date_completed'].count().index],
    y = [value for value in (df[df['date_completed'].notnull()].resample('M')['date_completed'].count() / 
                             df.resample('M')['wo_id'].count() * 100)],
    name = 'completed',
    marker = dict(color = '#87CD56'),
                )
trace2 = go.Bar(
    x = [date for date in df.resample('M')['wo_id'].count().index],
    y = [100 - value for value in (df[df['date_completed'].notnull()].resample('M')['date_completed'].count() / 
                                   df.resample('M')['wo_id'].count() * 100)],
    name = 'requested',
    marker = {'color': '#C70039'},
    
)

layout = go.Layout(
    hovermode = 'closest',
    title = 'Pct Work Orders Closed <i>vs</i><br>Opened Each Month',
    yaxis = {'title': 'percent'},
    xaxis = {'title': 'fiscal year'},
    barmode = 'stack'
                    )

fig = {'data': [trace1,trace2], 'layout': layout}
pyo.iplot(fig)

In [701]:
trace1 = go.Bar(
    x = [date for date in df[df['date_completed'].notnull()].resample('M')['date_completed'].count().index],
    y = [value for value in df[df['date_completed'].notnull()].resample('M')['date_completed'].count()],
    name = 'closed',
    marker = dict(color = '#87CD56'),
                )
trace2 = go.Bar(
    x = [date for date in df.resample('M')['wo_id'].count().index],
    y = [value for value in df.resample('M')['wo_id'].count()],
    name = 'requested',
    marker = {'color': '#C70039'},
    
)

layout = go.Layout(
    hovermode = 'closest',
    title = 'When are Work Orders Closed',
    yaxis = {'title': 'work orders<br>(per month)'},
    xaxis = {'title': 'fiscal year'},
    barmode = 'group'
                    )

fig = {'data': [trace1,trace2], 'layout': layout}
pyo.iplot(fig)

In [571]:
# # plot bar chart average duration of work orders closed and opened in same fiscal year
# top5 = list(df['prob_type'].value_counts().head(5).index)
# top5_df = df[(df['prob_type'].isin(top5))]
# yearly_avg_durations_top5 = {}
# for year in top5_df['fiscal_year_requested'].unique():
#     yearly_avg_durations_top5[year] = (top5_df[(top5_df['fiscal_year_requested'] == year) & 
#                                      (top5_df['fiscal_year_completed'] == year)]['duration'].mean())
    


# top10 = list(df['prob_type'].value_counts().head(10).index)
# top10_df = df[(df['prob_type'].isin(top10))]
# yearly_avg_durations_top10 = {}
# for year in top10_df['fiscal_year_requested'].unique():
#     yearly_avg_durations_top10[year] = (top10_df[(top10_df['fiscal_year_requested'] == year) & 
#                                      (top10_df['fiscal_year_completed'] == year)]['duration'].mean())
    

# top15 = list(df['prob_type'].value_counts().head(15).index)
# top15_df = df[(df['prob_type'].isin(top15))]
# yearly_avg_durations_top15 = {}
# for year in top15_df['fiscal_year_requested'].unique():
#     yearly_avg_durations_top15[year] = (top15_df[(top15_df['fiscal_year_requested'] == year) & 
#                                      (top15_df['fiscal_year_completed'] == year)]['duration'].mean())
    

# yearly_avg_durations = {}
# for year in df['fiscal_year_requested'].unique():
#     yearly_avg_durations[year] = (df[(df['fiscal_year_requested'] == year) & 
#                                      (df['fiscal_year_completed'] == year)]['duration'].mean())
    

# trace1 = go.Bar(
#     x = [year for year in yearly_avg_durations.keys()],
#     y = [value.days for value in yearly_avg_durations.values()],
#     name = 'all<br>work orders',
#     marker = {'color': '#3c5a89'},
#     text = ['All:<br>{} days'.format(value.days) for value in yearly_avg_durations.values()]
#                 )

# trace_top5 = go.Bar(
#     x = [year for year in yearly_avg_durations_top5.keys()],
#     y = [value.days for value in yearly_avg_durations_top5.values()],
#     name = 'top5<br>work orders',
#     marker = {'color': 'green'},
#     text = ['Top 5:<br>{} days'.format(value.days) for value in yearly_avg_durations_top5.values()]
#                 )

# trace_top10 = go.Bar(
#     x = [year for year in yearly_avg_durations_top10.keys()],
#     y = [value.days for value in yearly_avg_durations_top10.values()],
#     name = 'top10<br>work orders',
#     marker = {'color': 'brown'},
#     text = ['Top 10:<br>{} days'.format(value.days) for value in yearly_avg_durations_top10.values()]
#                 )

# trace_top15 = go.Bar(
#     x = [year for year in yearly_avg_durations_top15.keys()],
#     y = [value.days for value in yearly_avg_durations_top15.values()],
#     name = 'top15<br>work orders',
#     marker = {'color': 'orange'},
#     text = ['Top 15:<br>{} days'.format(value.days) for value in yearly_avg_durations_top15.values()]
#                 )

# trace2 = go.Scatter(
#     x = [year for year in df.groupby('fiscal_year_requested')['wo_id'].count().index],
#     y = [value for value in df.groupby('fiscal_year_requested')['wo_id'].count()],
#     mode = 'lines',
#     name = 'requests',
#     line = {'color': 'red',
#            'width': 3},
#     yaxis = 'y2')

# layout = go.Layout(
#     legend = {'orientation': 'h'},
#     hovermode = 'closest',
#     title = 'Average Work Order Durations<br>FY{}--FY{}'.format(min(yearly_avg_durations.keys()),
#                                                                max(yearly_avg_durations.keys())),
#     yaxis = {'title': 'days',
#             'showgrid': False,
#             'titlefont': {'color': '#3c5a89'},
#             'tickfont': {'color': '#3c5a89'}},
    
#     yaxis2 = {'title': 'work order<br>volume',
#              'showgrid': False,
#              'titlefont': {'color': 'red'},
#              'tickfont': {'color': 'red'},
#              'overlaying': 'y',
#              'side': 'right'},
    
#     plot_bgcolor = '#efefea',
#     paper_bgcolor = '#efefea'
    
#                     )
# fig = {'data':[trace1,trace_top5,trace_top10,trace_top15,trace2], 'layout':layout}

# pyo.iplot(fig)



## <font color=orange>Keeper</font>

In [695]:

# create dictionary of filtered dataframes for top problem types for each year
dataframe_filter_lists, filtered_dataframes = [],{}
yearly_avg_durations = {} 
for i, num in enumerate([3,5,10,15]):
    # create filter as list object and use loop counter to acess index of list filter
    # to create dataframe with the top X filter
    dataframe_filter_lists.append(df['prob_type'].value_counts().head(num).index)
    filtered_dataframes['top {}'.format(str(num))] = df[(df['prob_type'].isin(dataframe_filter_lists[i]))]
    
    for year in filtered_dataframes['top {}'.format(str(num))]['fiscal_year_requested'].unique():        
        yearly_avg_durations['top: {} year: {}'.format(num,year)] = (
            filtered_dataframes['top {}'.format(str(num))][(filtered_dataframes['top {}'.format(str(num))]['fiscal_year_requested'] == year) &
                                                          (filtered_dataframes['top {}'.format(str(num))]['fiscal_year_completed'] == year)]['duration'].mean())


# for comparison add key value for all problem types (not just top 3,5,10,15 as filtered above) 
for year in df['fiscal_year_requested'].unique():
    yearly_avg_durations['all {}'.format(year)] = (df[(df['fiscal_year_requested'] == year) & 
                                     (df['fiscal_year_completed'] == year)]['duration'].mean())

# trace for all work orders for each fiscal year
trace_all = go.Bar(
    x = [text.split()[-1] for text in yearly_avg_durations.keys()],
    y = [value.days for key,value in yearly_avg_durations.items() if 'all' in key],
    name = 'all<br>work orders',
    marker = {'color': '#3c5a89'},
    text = ['All:<br> {} days'.format(value.days) for key,value in yearly_avg_durations.items() if 'all' in key],
                )   
# traces for top 3, top 5, top 10, top 15 work orders per fiscal year
trace_topFilters = []
for num, color in zip([3,5,10,15],['#D3DEE4','#849DAB','#589ABF','#04A1F8']):
    trace_topFilters.append(go.Bar(
        x = [text.split()[-1] for text in yearly_avg_durations.keys()],
        y = [value.days for key,value in yearly_avg_durations.items() if 'top: {} year'.format(num) in key ],
        name = 'Top {}<br>work orders'.format(num),
        marker = {'color': color},
        text = ['Top {}:<br> {} days'.format(num,value.days) for key,value in yearly_avg_durations.items() if 'top: {} year'.format(num) in key],
                    )) 
# traces for annual work order reqeust volume   
trace_requestVolume = go.Scatter(
    x = [year for year in df.groupby('fiscal_year_requested')['wo_id'].count().index],
    y = [value for value in df.groupby('fiscal_year_requested')['wo_id'].count()],
    mode = 'lines',
    name = 'requests',
    line = {'color': '#C70039',
           'width': 3},
    yaxis = 'y2')

layout = go.Layout(
    legend = {'orientation': 'h'},
    hovermode = 'closest',
    title = 'Average Work Order Durations<br>FY{}--FY{}'.format(min(df['fiscal_year_requested']),
                                                                max(df['fiscal_year_requested'])),
    yaxis = {'title': 'days',
            'showgrid': False,
            'titlefont': {'color': '#3c5a89'},
            'tickfont': {'color': '#3c5a89'}},
    
    yaxis2 = {'title': 'work order<br>volume',
             'showgrid': False,
             'titlefont': {'color': '#C70039'},
             'tickfont': {'color': '#C70039'},
             'overlaying': 'y',
             'side': 'right'},
    
    plot_bgcolor = '#efefea',
    paper_bgcolor = '#efefea'
    
                    )
traces = trace_topFilters
for trace in [trace_all, trace_requestVolume]:
    traces.append(trace)


fig = {'data':traces, 'layout':layout}
pyo.iplot(fig)

In [ ]:
# box charts showing work order durations distribution by fiscal year

In [ ]:

# # create dictionary of filtered dataframes for top problem types for each year
# dataframe_filter_lists, filtered_dataframes = [],{}
# yearly_avg_durations = {} 
# for i, num in enumerate([3,5,10,15]):
#     # create filter as list object and use loop counter to acess index of list filter
#     # to create dataframe with the top X filter
#     dataframe_filter_lists.append(df['prob_type'].value_counts().head(num).index)
#     filtered_dataframes['top {}'.format(str(num))] = df[(df['prob_type'].isin(dataframe_filter_lists[i]))]
    
#     for year in filtered_dataframes['top {}'.format(str(num))]['fiscal_year_requested'].unique():        
#         yearly_avg_durations['top: {} year: {}'.format(num,year)] = (
#             filtered_dataframes['top {}'.format(str(num))][(filtered_dataframes['top {}'.format(str(num))]['fiscal_year_requested'] == year) &
#                                                           (filtered_dataframes['top {}'.format(str(num))]['fiscal_year_completed'] == year)]['duration'].mean())


# # for comparison add key value for all problem types (not just top 3,5,10,15 as filtered above) 
# for year in df['fiscal_year_requested'].unique():
#     yearly_avg_durations['all {}'.format(year)] = (df[(df['fiscal_year_requested'] == year) & 
#                                      (df['fiscal_year_completed'] == year)]['duration'].mean())

In [754]:

traces_duration = []
for year in df['fiscal_year_requested'].unique():
    traces_duration.append(go.Box(
        y = [duration.days for duration in df[(df['fiscal_year_requested'] == year) & 
                                              (df['fiscal_year_completed'] == year)]['duration']],
        name = 'FY {}'.format(year),
        boxpoints = 'all',
        jitter = .5,
        pointpos = -1.8)
                          )
layout = {
        'title': 'Work Order Durations',
        'plot_bgcolor' : '#efefea',
        'paper_bgcolor' : '#efefea'
        }
fig = {'data':traces_duration, 'layout':layout}

pyo.iplot(fig)

In [751]:
data = []
for year in df['fiscal_year_requested'].unique():
    trace = {
        'type': 'violin',
        'y': [duration.days for duration in df[(df['fiscal_year_requested'] == year) & 
                                          (df['fiscal_year_completed'] == year)]['duration']],
        'box': {'visible': True},
        'meanline': {'visible': True,
                    'color':'red',
                    'width': 7},
        #'line': {'color':'blue'},
        'fillcolor': '#8CA8C9',
        'opacity': .2,
        'x0': 'FY {}'.format(year),
        'name': 'FY {}'.format(year),
            }
    data.append(trace)
                
layout = {'title': 'Work Order Duration<br> Distribution by Fiscal Year',
         'yaxis': {'zeroline': False}}

fig = {'data': data, 'layout':layout}
# fig = {
    
#     'layout': {
#         'title': 'Work Order Duration<br> Distribution by Fiscal Year',
#         'yaxis': {'zeroline': False}
#     }
#         }


pyo.iplot(fig)